In [1]:
# !pip install gigachain faiss-cpu sentence-transformers sentencepiece rank_bm25 datasets --quiet

In [2]:
import datasets
from langchain.docstore.base import Document
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import EnsembleRetriever, BM25Retriever

/Users/19814197/Documents/gigachain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Данные

In [3]:
ds = datasets.load_dataset("sberquad")
ds["train"][0]

{'id': 62310,
 'title': 'SberChallenge',
 'context': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.',
 'question': 'чем представлены органические остатки?',
 'answers': {'text': ['известковыми выделениями сине-зелёных водорослей'],
  'answer_start': [109]}}

In [4]:
validation_ds = ds["validation"]
documents = [
    Document(page_content=context)
    for context in set(validation_ds["context"])
]

## Ретривал

### Эмбеддинг модель

In [5]:
from typing import List, Coroutine, Any


class HuggingFaceE5Embeddings(HuggingFaceEmbeddings):
    def embed_query(self, text: str) -> List[float]:
        text = f"query: {text}"
        return super().embed_query(text)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = [f"passage: {text}" for text in texts]
        return super().embed_documents(texts)

    async def aembed_query(self, text: str) -> Coroutine[Any, Any, List[float]]:
        text = f"query: {text}"
        return await super().aembed_query(text)

    async def aembed_documents(self, texts: List[str]) -> Coroutine[Any, Any, List[List[float]]]:
        texts = [f"passage: {text}" for text in texts]
        return await super().aembed_documents(texts)

In [6]:
embedding = HuggingFaceE5Embeddings(model_name="intfloat/multilingual-e5-base")

In [7]:
faiss_db = FAISS.from_documents(documents, embedding=embedding)

In [8]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 5})

In [9]:
validation_ds = validation_ds.map(
    lambda x: {
        "embedding_retrieved": [
            passage.page_content
            for passage in embedding_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [08:19<00:00, 10.09 examples/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved'],
    num_rows: 5036
})

In [10]:
def acc_top(dataset: datasets.Dataset, right_column: str, answer_column: str) -> float:
    temp_dataset = dataset.map(
        lambda x: {
            "is_right_retrieved": x[right_column] in x[answer_column]
        }
    )
    return sum(temp_dataset["is_right_retrieved"]) / len(temp_dataset)

In [11]:
acc_top(validation_ds, "context", "embedding_retrieved")

Map: 100%|██████████| 5036/5036 [00:01<00:00, 3633.59 examples/s]


0.9116362192216044

### BM25

In [12]:
import string


def tokenize(s: str) -> list[str]:
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")

In [13]:
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=5,
)

In [14]:
validation_ds = validation_ds.map(
    lambda x: {
        "bm25_retrieved": [
            passage.page_content
            for passage in bm25_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [06:27<00:00, 12.99 examples/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved', 'bm25_retrieved'],
    num_rows: 5036
})

In [15]:
acc_top(validation_ds, "context", "bm25_retrieved")

Map: 100%|██████████| 5036/5036 [00:01<00:00, 2714.89 examples/s]


0.9197776012708498

### Ансамбль

In [16]:
embedding_retriever = faiss_db.as_retriever(search_kwargs={"k": 2})
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    preprocess_func=tokenize,
    k=3,
)

In [17]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[embedding_retriever, bm25_retriever],
    weights=[0.4, 0.6],
)

In [18]:
validation_ds = validation_ds.map(
    lambda x: {
        "retrieved": [
            passage.page_content
            for passage in ensemble_retriever.get_relevant_documents(x["question"])
        ]
    }
)
validation_ds

Map: 100%|██████████| 5036/5036 [21:30<00:00,  3.90 examples/s] 


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'embedding_retrieved', 'bm25_retrieved', 'retrieved'],
    num_rows: 5036
})

In [19]:
acc_top(validation_ds, "context", "retrieved")

Map: 100%|██████████| 5036/5036 [00:01<00:00, 2806.17 examples/s]


0.9692216044479746

## E2E решение

In [20]:
from langchain.chains import RetrievalQA
from langchain.llms.gigachat import GigaChat

In [33]:
llm = GigaChat(credentials=...)

In [36]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever,
    return_source_documents=True,
)

In [37]:
qa.invoke({"query": "Что такое вода?"})

{'query': 'Что такое вода?',
 'result': 'Вода — это бинарное неорганическое соединение, состоящее из двух атомов водорода и одного атома кислорода, соединенных ковалентной связью. Она является прозрачной жидкостью без цвета, запаха и вкуса при нормальных условиях. Вода может существовать в различных состояниях, включая жидкое, твердое и газообразное. Она играет важную роль в жизни на Земле и составляет примерно 0,05% ее массы.',
 'source_documents': [Document(page_content='На самом деле из-за низкого давления вода не может существовать в жидком состоянии на большей части (около 70 %) поверхности Марса. Вода в состоянии льда была обнаружена в марсианском грунте космическим аппаратом НАСА Феникс . В то же время собранные марсоходами Спирит и Opportunity геологические данные позволяют предположить, что в далёком прошлом вода покрывала значительную часть поверхности Марса. Наблюдения в течение последнего десятилетия позволили обнаружить в некоторых местах на поверхности Марса слабую гейзер